In [4]:
import pandas as pd

# 1. 데이터 로드
matching_df = pd.read_csv('matching_questions_cnn_953.csv')
prices_df = pd.read_csv('question_prices_with_changes_cnn.csv')

# 2. 필요한 열 선택
prices_subset = prices_df[['yes_price', 'change_1d', 'change_3d', 'change_5d']]

# 3. 열 옆으로 붙이기
# axis=1을 사용해 수평으로 결합
combined_df = pd.concat([matching_df, prices_subset], axis=1)

# 4. 결과를 새로운 CSV로 저장
combined_df.to_csv('cnn_with_labels.csv', index=False)
print("저장되었습니다.")

저장되었습니다.


In [15]:
import pandas as pd

# Load the main data
combined_df = pd.read_csv("combined_labeled_data.csv")

# Drop the specified columns from combined_labeled_data.csv
columns_to_drop = ['change_1d', 'change_3d', 'change_5d', 'text']
combined_df = combined_df.drop(columns=columns_to_drop, errors='ignore')

# Load the question descriptions
closed_questions_df = pd.read_csv("closed_questions_description.csv")

# Merge the two DataFrames on 'matching_questions' and 'question'
merged_df = pd.merge(combined_df, closed_questions_df, left_on='matching_questions', right_on='question', how='inner')

# Drop 'question', 'description', 'end_date' columns after merge
columns_to_drop_after_merge = ['question', 'description']
merged_df = merged_df.drop(columns=columns_to_drop_after_merge, errors='ignore')

# Save the result to a new CSV (optional)
merged_df.to_csv("merged_result.csv", index=False)


In [16]:
import pandas as pd
from datetime import datetime

# CSV 불러오기
df = pd.read_csv("merged_result.csv")

# 날짜 차이 계산 함수 (문자열 그대로 두고 계산만)
def date_diff_more_than_5(upload_str, end_str):
    try:
        upload_dt = datetime.strptime(upload_str, "%Y-%m-%dT%H:%M:%SZ")
        end_dt = datetime.strptime(end_str, "%Y-%m-%dT%H:%M:%SZ")
        return (end_dt - upload_dt).days > 5
    except:
        return False  # 파싱 오류가 있으면 삭제 대상

# 필터링 적용 (5일 초과만 남김)
df = df[df.apply(lambda row: date_diff_more_than_5(row["upload_date"], row["end_date"]), axis=1)]

# 저장 (형식 유지됨)
df.to_csv("merged_result.csv", index=False)
print("날짜 형식을 유지하며 5일 이하 데이터는 제거되었습니다.")



날짜 형식을 유지하며 5일 이하 데이터는 제거되었습니다.


In [1]:
import pandas as pd

# 1. 두 CSV 파일 로드
df_fox = pd.read_csv("matching_questions_politics_economy_fox.csv")
df_general = pd.read_csv("matching_questions_politics_economy.csv")

# 2. 두 데이터프레임 병합
df_combined = pd.concat([df_fox, df_general], ignore_index=True)

# 3. 결과 확인 (선택적)
print("병합된 데이터프레임의 첫 몇 행:")
print(df_combined.head())
print(f"총 행 수: {len(df_combined)}")

# 4. 병합된 데이터를 새로운 CSV 파일로 저장
df_combined.to_csv("combined_labeled_data.csv", index=False)
print("병합된 파일이 'combined_labeled_data.csv'로 저장되었습니다.")

병합된 데이터프레임의 첫 몇 행:
                                               title  \
0  Gutfeld: We're seeing the 'downfall of the ina...   
1  Gutfeld: We're seeing the 'downfall of the ina...   
2  Trump takes lead on VP Harris, top pollster re...   
3  'She never knew?': Harris under fire for claim...   
4    Jesse Watters: Kamala just does what she's told   

                                           url                upload_date  \
0  https://www.youtube.com/watch?v=6DmwFanCzfs  2024-10-24 22:27:37+00:00   
1  https://www.youtube.com/watch?v=6DmwFanCzfs  2024-10-24 22:27:37+00:00   
2  https://www.youtube.com/watch?v=Jt8UgIdBU4s  2024-10-24 15:30:09+00:00   
3  https://www.youtube.com/watch?v=AZg9NMY4UnU  2024-10-24 07:30:06+00:00   
4  https://www.youtube.com/watch?v=6ZkuOWvx9Pw  2024-10-24 01:13:43+00:00   

                                  matching_questions  similarity  \
0                     Kamala Harris blowout victory?    0.647121   
1              Kamala Harris fires campaign m

In [2]:
import pandas as pd

# CSV 파일 불러오기
closed_questions = pd.read_csv('closed_questions_politics_economy.csv')
combined_data = pd.read_csv('combined_labeled_data.csv')

# 1. 중복 확인: closed_questions에서 question의 고유성 확인
print("closed_questions에서 question의 고유 개수:", closed_questions['question'].nunique())
print("closed_questions의 총 행 수:", len(closed_questions))
if closed_questions['question'].duplicated().any():
    print("중복된 question이 존재합니다. 중복된 항목:")
    print(closed_questions[closed_questions['question'].duplicated(keep=False)][['question', 'condition_id']])

# 2. closed_questions에서 중복 제거 (필요한 경우)
# question이 고유해야 하므로, 중복된 경우 첫 번째 항목만 유지하거나 다른 기준으로 선택
closed_questions_unique = closed_questions.drop_duplicates(subset=['question'], keep='first')

# 3. 병합 수행
merged_data = pd.merge(
    combined_data,
    closed_questions_unique[['question', 'condition_id']],
    how='left',
    left_on='matching_questions',
    right_on='question'
)

# 4. 병합 후 행 수 확인
print("원본 combined_data 행 수:", len(combined_data))
print("병합 후 merged_data 행 수:", len(merged_data))

# 5. 중복된 question 열 제거
merged_data = merged_data.drop(columns=['question'], errors='ignore')

# 6. 결과 저장
merged_data.to_csv('combined_labeled_data_updated.csv', index=False)

print("병합 완료. 업데이트된 파일이 'combined_labeled_data_updated.csv'로 저장되었습니다.")

closed_questions에서 question의 고유 개수: 7652
closed_questions의 총 행 수: 7715
중복된 question이 존재합니다. 중복된 항목:
                                               question  \
519                      Will Biden drop out by Friday?   
558                      Will Biden drop out by Friday?   
1624            Who is asked the first debate question?   
1807           Will Joe Biden make an appearance today?   
1819           Will Joe Biden make an appearance today?   
...                                                 ...   
7575  Will Karoline Leavitt say "Crypto" or "Bitcoin...   
7585  Will Karoline Leavitt say "President" 80+ time...   
7624  Will Karoline Leavitt say "Executive Order" du...   
7627  Will Karoline Leavitt say "President" 100+ tim...   
7683  Fear & Greed Index says "Extreme Fear" on Friday?   

                                           condition_id  
519   0xe448c3f389cef8a227af746c20a739cbbffa496b0c26...  
558   0x05cf99a3c7ee91540f484f8c7b5602e9db141cd8d580...  
1624  0x156a37a9f